In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del__4_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

2567282

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

19063975

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1707290662
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 128364034


b'chr22\t4\tC\tM\t+\nchr22\t101\tC\tS\t+\nchr22\t1389\tG\t-\t-\nchr22\t2083\tA\tW\t+\nchr22\t2312\tG\tR\t+\nchr22\t6282\tT\tW\t+\nchr22\t7343\tG\tA\t-\nchr22\t9596\tA\t-\t-\nchr22\t10013\tG\tR\t+\nchr22\t10749\t-\tGGA\t-\nchr22\t12953\tA\tM\t+\nchr22\t13235\tT\tK\t+\nchr22\t13891\tG\tA\t-\nchr22\t15670\tT\tG\t-\nchr22\t16360\tA\tM\t+\nchr22\t16439\tC\tG\t-\nchr22\t18079\tT\t-\t-\nchr22\t19319\tA\tW\t+\nchr22\t19529\tG\tR\t+\nchr22\t19631\tG\tR\t+\nchr22\t23368\tA\tR\t+\nchr22\t23832\tG\tC\t-\nchr22\t24134\tA\tG\t-\nchr22\t24398\tT\tG\t-\nchr22\t25176\tC\tS\t+\nchr22\t25362\tG\tC\t-\nchr22\t26402\tC\tS\t+\nchr22\t26733\tT\tY\t+\nchr22\t30498\t-\tC\t+\nchr22\t30663\tA\tC\t-\nchr22\t32536\tTA\t-\t-\nchr22\t33373\tC\tS\t+\nchr22\t33663\tT\tY\t+\nchr22\t34334\t-\tT\t+\nchr22\t35596\tC\tA\t-\nchr22\t35736\tA\tC\t-\nchr22\t36209\tG\tS\t+\nchr22\t38756\tT\tC\t-\nchr22\t39851\t-\tA\t+\nchr22\t41369\tA\tW\t+\nchr22\t41905\tA\tC\t-\nchr22\t42830\tG\tA\t-\nchr22\t45654\tT\tW\t+\nchr22\t46657\tC\tY

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 23 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

Problem with fit: negative mean. Using mean and std instead fitting parameters!
Problem with fit: negative mean. Using mean and std instead fitting parameters!
Problem with fit: Runtime Error. Using mean and std instead fitting parameters!


In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

deletion	chr22:1-24866500	24866500	0.1790	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0002	0.6555	0
deletion	chr22:24867401-26772500	1905100	0.5124	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0002	0.0000	4258100
duplication	chr22:28896401-46831000	17934600	1.4528	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	3533900
deletion	chr22:46831001-51304600	4473600	0.5984	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0246	0
deletion	chr22:1-26773000	26773000	0.2030	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0002	0.6088	0
duplication	chr22:28896001-46831000	17935000	1.4443	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	3533900
deletion	chr22:46831001-51305000	4474000	0.6038	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0247	0
deletion	chr22:1-26770000	26770000	0.2043	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0002	0.6089	0
duplication	chr22:28890001-46830000	17940000	1.4273	0.000000e+00

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,24866500,24866500,0.17897079063481733,0.00022847302216511128
deletion,chr22,24867401,26772500,1905100,0.5124057931510957,0.0001757388650642083
duplication,chr22,28896401,46831000,17934600,1.4528042644348398,7.56526028455634e-05
deletion,chr22,46831001,51304600,4473600,0.5984343449938964,0.0001366045389519194


In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,26773000,26773000,0.20302493687105533,0.00021883051092075765
duplication,chr22,28896001,46831000,17935000,1.4443000601026115,7.565132416404729e-05
deletion,chr22,46831001,51305000,4474000,0.6038287883088232,0.0001366045389519194


In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,26770000,26770000,0.20431960839146343,0.00021890251805258674
duplication,chr22,28890001,46830000,17940000,1.4273446657349096,7.563681969109218e-05
deletion,chr22,46830001,51310000,4480000,0.6091707256882356,0.0001365416483059526


In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,26800000,26800000,0.2098283179473571,0.00021824924975480457
duplication,chr22,28900001,46800000,17900000,1.4321525575874527,7.566658264490581e-05
deletion,chr22,46800001,51400000,4600000,0.6362175508899787,0.00013578157052704677


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,16000000,16000000,0.0,-1


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__4_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app